In [57]:
import numpy as np                            #Library that has extensive set of math methods and functions.
from scipy.integrate import odeint            #Library that has function for solving differentail equations
from scipy.interpolate import interp1d        #Library containing fuctions of interpolating data
from scipy.optimize import curve_fit          #Library containing fuctions of fitting paramaters to data
from scipy.stats import t as tdist
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import math
from itertools import repeat
from datetime import timedelta
import csv



#Library to plot data
import plotly.graph_objects as go

#använd curvefit för att anpassa parametrar 

mobility_data = pd.read_csv('https://raw.githubusercontent.com/Dan-Irl/MVEX-Matematiska-modeller-f-r-smittspridning/main/SwedenMobilityData.csv')   #imports mobility data from github

temperature_data = pd.read_json('https://raw.githubusercontent.com/Dan-Irl/MVEX-Matematiska-modeller-f-r-smittspridning/main/smhidata_2020_2021.json')  #imports temperature data from github 

admission_data = pd.read_json('https://raw.githubusercontent.com/Dan-Irl/MVEX-Matematiska-modeller-f-r-smittspridning/main/Inskrivningsdata.json')                  #imports admission data from github

VT_mobility_data = pd.read_csv('https://raw.githubusercontent.com/Dan-Irl/MVEX-Matematiska-modeller-f-r-smittspridning/main/VT_mobility.csv')


#Jämför RMSE mellan modeller berode på antal datapunkter
#jämför hur ofta prediction hamnar i predictionsintervall mellan modeller berode på antal datapunkter

In [108]:
# A array of time points in days for the model to use
prediction_time = 173
t = np.linspace(1, prediction_time, prediction_time)

#Defines how many data point are to be used for model fitting
fittingPoints = 173 # Max 173 days
t_fittingdata = np.linspace(0, fittingPoints, fittingPoints)


dailyAdmissions = admission_data['Inskrivningar']


Created the mobility function for Västtrafik

In [84]:
temp_VT = VT_mobility_data['Totalt']

T = []
for week in range(len(temp_VT)-1):
    T.extend(np.linspace(temp_VT[week],temp_VT[week+1],8)[:7])
    
VT_mobilityfunc = interp1d(t,T[:prediction_time],kind='linear',fill_value="extrapolate")

functionfig = go.Figure()

functionfig.add_trace(go.Scatter(x=t, y=VT_mobilityfunc(t),
                    mode='lines',
                    name='Mobility function'))
functionfig.update_layout(
                title='Mobility functions',
                xaxis_title='Weeks',
                yaxis_title='Weekly change in mobility %',
                autosize=False,
                width=1000,
                
                font=dict(
                family="Courier New, monospace",
                size=14,
                color="RebeccaPurple"
    )
) 
functionfig.show()

This cell defines the model function and contains most of its dependensies. The SIR model is defined as presendted my Wiess (STORT FEL Gamma är 0.2 vilket är för dagar inte 

In [109]:
#Weiss implementering 

# Contact rate, transmisabilty, beta, and mean recovery rate gamma.
#nunmpy quantile för confidensintervall för beta
#försjkut admissions
#Lagen borde inte vara i modelfunction, för den modellerar antalet infected, lagen borde bara vara på predicted admissions

# Total population, N.
N = 700000 #göteborg stad
# Initial number of infected and recovered individuals, I0 and R0.
I0, R0 = 1, 0
# Everyone else, S0, is susceptible to infection initially.
S0 = N - I0 - R0

tau = 1         #transmisability chance
gamma =  0.2   #recovery rate 1/days
lag = 21


# Initial conditions vector
y0 = S0, I0, R0
p = 0.013 #fraction of the present number of infectious individuals

def modelfunction(t,c0,c1): #Enbedded function that takes 
  def betafunc(t):
    kappa =  c0+c1*VT_mobilityfunc(t)
    b = kappa*tau
    beta = b/N
    return beta

  # The SIR model differential equations.
  def deriv(y, t, N):
    S, I, R = y
    dSdt = -betafunc(t-lag) * S * I 
    dIdt = betafunc(t-lag) * S * I - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt
  
  result = odeint(deriv, y0, t, args=(N,))
  S, I, R = result.T
  #Returerar totalt antal inskrivningar varje dag
  return I*p



#Curve fit function that takes the modelfunction with the given parameters to optimize and returns them with optimal value and covariance
popt, pcov = curve_fit(modelfunction, t_fittingdata, dailyAdmissions[:fittingPoints], p0=[0.4198,0.003781])

#best VT guess p0=[0.4198,0.003781])
#Best google guess p0=[3,0.6])

print(popt)
print(pcov)
print(mean_squared_error(dailyAdmissions[0:prediction_time],modelfunction(t,*popt), squared = False))

fig = go.Figure() #Creates a plot using graph objects and then adds each graph to figure
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=fittingPoints)), 
                    y=dailyAdmissions,
                    mode='markers',
                    name='Admission data'))
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=prediction_time)), 
                    y=modelfunction(t,*popt),
                    mode='lines',
                    name='Predicted admissions'))
fig.update_layout(
                    title='Time dependant SIR model',
                    xaxis_title='Weeks',
                    yaxis_title='Admissions',
                    autosize=False,
                    width=1000,
                    
                    font=dict(
                    family="Courier New, monospace",
                    size=14,
                    color="RebeccaPurple"
    )
)
fig.show()

[0.4198   0.003781]
[[3.29666948e-06 5.99101111e-08]
 [5.99101111e-08 1.81675581e-08]]
120.332638366


Plots the predicted admissions and the admission data, for comparison

Plots confidence intervals of model

In [110]:
numBetas = 10 #number of betas used for normal dist for conf. intervalls
numEpsilon = 20

##################  Confidence intervall calculation
confint = np.zeros([prediction_time,2]) #defines confidence intervall arrays


# Generates an array of betas using normal dist.
betas=np.random.multivariate_normal(popt,pcov,numBetas) 
values = np.zeros((prediction_time,numBetas))

for beta in range(len(betas)): 
  values[:,beta] = (modelfunction(t,*betas[beta]))
  
for day in range(len(t)):
  confint[day,0]=np.quantile(values[day,:],0.025)
  confint[day,1]=np.quantile(values[day,:],0.975)
  

##################  Prediction intervall calculation
#få med beta skattning i predictionsintervall, dvs generera beta och fel för varje dag 

std_residual = np.std(modelfunction(t_fittingdata,*popt)-dailyAdmissions[:fittingPoints], ddof=2) #calcuales standard deviation of residual between model and data
predint = np.zeros([prediction_time,2])

temp = []
pred_matrix = np.zeros((prediction_time,(numBetas*numEpsilon)))
for day in range(prediction_time): #loops trough model values each day a adds the error values generated from t-dist to each prediction 
  for beta in range(len(betas)):
      temp =  np.add(modelfunction(t,*betas[beta])[day],tdist.rvs(prediction_time,0,std_residual,numEpsilon))
      pred_matrix[day,(beta*numEpsilon):(beta*numEpsilon+numEpsilon)] = temp

for day in range(prediction_time): #calculates the prediction intervals
    predint[day,0] = np.quantile(pred_matrix[day],0.025)
    predint[day,1] = np.quantile(pred_matrix[day],0.975)
    
predint = np.clip(predint,0,None) #removes all negative values from lover prediction interval

fig = go.Figure() #Creates a plot using graph objects and then adds each graph to figure
fig.add_trace(go.Scatter( #plots the fitting admission data points
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=fittingPoints)), 
                    y=dailyAdmissions,
                    mode='markers',
                    name='Fitting admission points'))
fig.add_trace(go.Scatter( #plots all the available admission data points
                    x=pd.Series(pd.date_range(pd.to_datetime('2020-03-01')+timedelta(days=fittingPoints), freq="D", periods=prediction_time-fittingPoints)),
                    y=dailyAdmissions[fittingPoints:],
                    mode='markers',
                    marker=dict(
                        color='firebrick',
                    ),
                    name='Admission data'))
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=prediction_time)), 
                    y=modelfunction(t,*popt),
                    mode='lines',
                    name='Predicted admissions'))
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=prediction_time)), 
                    y=confint[:,0],
                    mode='lines',
                    name='Lower confidence interval',
                    line = dict( width=2, dash='dash')))
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=prediction_time)), 
                    y=confint[:,1],
                    mode='lines',
                    name='Upper confidence interval',
                    line = dict(width=2, dash='dash')))
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=prediction_time)), 
                    y=predint[:,0],
                    mode='lines',
                    name='Lower prediction interval',
                    line = dict(width=2, dash='dot')))
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=prediction_time)), 
                    y=predint[:,1],
                    mode='lines',
                    name='Upper prediction interval',
                    line = dict(width=2, dash='dot')))
fig.add_annotation(x=pd.to_datetime("2020-03-01"),y=0, 
                   text=f"RMSE: {math.floor(mean_squared_error(dailyAdmissions[:prediction_time],modelfunction(t,*popt), squared = False))}",
                   showarrow=False,
                   bordercolor="black",
                   xshift=48,
                   yshift=270
                  )
fig.update_layout(
                    title='Västtrafik mobility based SIR model',
                    xaxis_title='Date',
                    yaxis_title='Admissions',
                    autosize=False,
                    width=1000,
                    xaxis_range=["2020-03-01",pd.to_datetime('2020-03-01')+timedelta(days=prediction_time)],
                    
                    font=dict(
                      family="Courier New, monospace",
                      size=14,
                      color="RebeccaPurple"
                    )
)
fig.show()

Cell calcualtes the cumulative admissions and confidens/prediktion intervalls

In [106]:
numBetas = 10 #number of betas used for normal dist for conf. intervalls
numEpsilon = 100

##################  Confidence intervall calculation
confint = np.zeros([prediction_time,2]) #defines confidence intervall arrays


# Generates an array of betas using normal dist.
betas=np.random.multivariate_normal(popt,pcov,numBetas) 
values = np.zeros((prediction_time,numBetas))

for beta in range(len(betas)): 
  values[:,beta] = np.cumsum(modelfunction(t,*betas[beta]))
  
for day in range(len(t)):
  confint[day,0]=np.quantile(values[day,:],0.025)
  confint[day,1]=np.quantile(values[day,:],0.975)
  

##################  Prediction intervall calculation
#få med beta skattning i predictionsintervall, dvs generera beta och fel för varje dag 

std_residual = np.std(modelfunction(t_fittingdata,*popt)-dailyAdmissions[:fittingPoints], ddof=2) #calcuales standard deviation of residual between model and data
predint = np.zeros([prediction_time,2])

temp = []
pred_matrix = np.zeros((prediction_time,(numBetas*numEpsilon)))
for day in range(prediction_time): #loops trough model values each day a adds the error values generated from t-dist to each prediction 
  for beta in range(len(betas)):
      temp =  np.add(np.cumsum(modelfunction(t,*betas[beta]))[day],tdist.rvs(prediction_time,0,std_residual,numEpsilon))
      pred_matrix[day,(beta*numEpsilon):(beta*numEpsilon+numEpsilon)] = temp

for day in range(prediction_time): #calculates the prediction intervals
    predint[day,0] = np.quantile(pred_matrix[day],0.025)
    predint[day,1] = np.quantile(pred_matrix[day],0.975)
    
predint = np.clip(predint,0,None) #removes all negative values from lover prediction interval

fig = go.Figure() #Creates a plot using graph objects and then adds each graph to figure
fig.add_trace(go.Scatter( #plots the fitting admission data points
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=fittingPoints)), 
                    y=np.cumsum(dailyAdmissions),
                    mode='markers',
                    name='Fitting admission points'))
fig.add_trace(go.Scatter( #plots all the available admission data points
                    x=pd.Series(pd.date_range(pd.to_datetime('2020-03-01')+timedelta(days=fittingPoints), freq="D", periods=prediction_time-fittingPoints)),
                    y=np.cumsum(dailyAdmissions)[fittingPoints:],
                    mode='markers',
                    marker=dict(
                        color='firebrick',
                    ),
                    name='Admission data'))
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=prediction_time)), 
                    y=np.cumsum(modelfunction(t,*popt)),
                    mode='lines',
                    name='Predicted cumulative admissions'))
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=prediction_time)), 
                    y=confint[:,0],
                    mode='lines',
                    name='Lower confidence interval',
                    line = dict( width=2, dash='dash')))
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=prediction_time)), 
                    y=confint[:,1],
                    mode='lines',
                    name='Upper confidence interval',
                    line = dict(width=2, dash='dash')))
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=prediction_time)), 
                    y=predint[:,0],
                    mode='lines',
                    name='Lower prediction interval',
                    line = dict(width=2, dash='dot')))
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=prediction_time)), 
                    y=predint[:,1],
                    mode='lines',
                    name='Upper prediction interval',
                    line = dict(width=2, dash='dot')))
fig.add_annotation(x=pd.to_datetime("2020-03-01"),y=0, 
                   text=f"RMSE: {math.floor(mean_squared_error(dailyAdmissions[:prediction_time],modelfunction(t,*popt), squared = False))}",
                   showarrow=False,
                   bordercolor="black",
                   xshift=48,
                   yshift=270
                  )
fig.update_layout(
                    title='Västtrafik mobility based SIR model',
                    xaxis_title='Date',
                    yaxis_title='Admissions',
                    autosize=False,
                    width=1000,
                    xaxis_range=["2020-03-01",pd.to_datetime('2020-03-01')+timedelta(days=prediction_time)],
                    
                    font=dict(
                      family="Courier New, monospace",
                      size=14,
                      color="RebeccaPurple"
                    )
)
fig.show()

In [111]:
#calcualtes RMSE for 21 days ahead of the fitting data as it increases with

header = ['Idex', 'Fitting points', 'MAPE'] #header for csv file
RMSEovertime = []   #init RMSE array
startdata = 42  #number of intial data points for MAP calculation in days


#writes a csv file of Index, number of fitting points and MAP
with open('SIR_MAPE_daily.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)
    
    #loops over all data point for fitting and curve fits the model and calculates MAPE
    for day in range(len(dailyAdmissions)):
        #breaks loop when reaching final value
        if day+startdata+21 == len(dailyAdmissions):
            break
        
        #inits temp array for writing to csv file and time array for model
        temp_RMSEovertime = []
        t_MAPE1 = np.linspace(0,day+startdata,day+startdata)
        t_MAPE2 = np.linspace(0,day+startdata+21,day+startdata+21)
        
        #curve fits model to fitting data
        MAPE_popt, pcov = curve_fit(modelfunction, t_MAPE1, dailyAdmissions[:day+startdata], p0=[0.4198,0.003781])
        
        #creates temp list of Index, number of fitting points and MAP
        temp_RMSEovertime.append(day)
        temp_RMSEovertime.append(day+startdata-21)
        temp_RMSEovertime.append(mean_absolute_percentage_error(np.cumsum(dailyAdmissions[:day+startdata+21])[-21:],np.cumsum(modelfunction(t_MAPE2,*MAPE_popt))[-21:]))
        
        #Appends matrix of all MAP vallues
        RMSEovertime.append(temp_RMSEovertime)
        
         # write the data
        writer.writerow(temp_RMSEovertime)

Cell that seatches for a global minimum of the paramaters given using RMSE

In [ ]:
#VT global param search
paramC0 = np.linspace(0,4,50)  #x-values
paramC1 = np.linspace(0,1,50)  #y-values

# #Google global param search
# params.append(np.linspace(0,4,50)) 
# params.append(np.linspace(0,1,50))

result = np.zeros((len(paramC0),len(paramC1)))
for param1 in range(len(paramC0)):
    for param0 in range(len(paramC1)):
        result[param0,param1] = math.log(mean_squared_error(dailyAdmissions,modelfunction(t,paramC0[param0],paramC1[param1]), squared = False))



#Calcualtes the log of RMSE for each point in parameter room
# for input in inputs:
#     result.append(math.log(mean_squared_error(dailyAdmissions,modelfunction(t,*input), squared = False)))

In [ ]:
fig = go.Figure() #Creates a plot using graph objects and then adds each graph to figure
# fig.add_trace(go.Scatter(x=t, y=dailyAdmissions,
#                     mode='lines',
#                     name='Admission data'))
# fig.add_trace(go.Scatter(x=t, y=modelfunction(t,*best[0]),
#                     mode='lines',
#                     name='Predicted admissions'))

fig.add_contour(
                    z=result,
                    x=paramC0,
                    y=paramC1,
                    colorbar=dict(
                        titleside='right',
                        title='Log RMSE', # title here
                    )
)  
  

fig.update_layout(
                    title='Global parameter search Google mobility data',
                    xaxis_title='c0 value',
                    yaxis_title='c1 value',
                    autosize=False,
                    width=800,
                    height=800,
                    font=dict(
                        family="Courier New, monospace",
                        size=18,
                        color="RebeccaPurple"
    )
)
fig.show()

#Countour plot plotta log(RMSE) för Z axeln

